# L5: Automate Event Planning

In this lesson, you will learn more about Tasks.

The libraries are already installed in the classroom. If you're running this notebook on your own machine, you can install the following:
```Python
!pip install crewai==0.28.8 crewai_tools==0.1.6 langchain_community==0.0.29
```

In [1]:
# Warning control
import warnings
warnings.filterwarnings('ignore')

- Import libraries, APIs and LLM

In [2]:
from crewai import Agent, Task, Crew
import os
from langchain_openai import ChatOpenAI

**Note**: 
- The video uses `gpt-4-turbo`, but due to certain constraints, and in order to offer this course for free to everyone, the code you'll run here will use `gpt-3.5-turbo`.
- You can use `gpt-4-turbo` when you run the notebook _locally_ (using `gpt-4-turbo` will not work on the platform)
- Thank you for your understanding!

In [ ]:
os.environ["OPENAI_API_KEY"] = 'dummy_key'
os.environ["SERPER_API_KEY"]  = 'xxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxx' # you can get free serper api keys from serper.dev. Just sign up and copy here.

llm = ChatOpenAI(
    model = 'ollama/llama3.3:latest',
    base_url="http://localhost:11434/v1"
)

## crewAI Tools

In [4]:
from crewai_tools import ScrapeWebsiteTool, SerperDevTool

# Initialize the tools
search_tool = SerperDevTool()
scrape_tool = ScrapeWebsiteTool()

/home/taher490/Desktop/crew_ai/myenv/lib/python3.12/site-packages/pydantic/fields.py:1093: PydanticDeprecatedSince20: Using extra keyword arguments on `Field` is deprecated and will be removed. Use `json_schema_extra` instead. (Extra keys: 'required'). Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.11/migration/
  warn(


## Creating Agents

In [5]:
# Agent 1: Venue Coordinator
venue_coordinator = Agent(
    role="Venue Coordinator",
    goal="Identify and book an appropriate venue "
    "based on event requirements",
    tools=[search_tool, scrape_tool],
    verbose=True,
    backstory=(
        "With a keen sense of space and "
        "understanding of event logistics, "
        "you excel at finding and securing "
        "the perfect venue that fits the event's theme, "
        "size, and budget constraints."
    ),
    llm=llm
)

In [ ]:
 # Agent 2: Logistics Manager
logistics_manager = Agent(
    role='Logistics Manager',
    goal=(
        "Manage all logistics for the event "
        "including catering and equipment"
    ),
    tools=[search_tool, scrape_tool],
    verbose=True,
    backstory=(
        "Organized and detail-oriented, "
        "you ensure that every logistical aspect of the event "
        "from catering to equipment setup "
        "is flawlessly executed to create a seamless experience."
    ),
    llm=llm
)

In [7]:
# Agent 3: Marketing and Communications Agent
marketing_communications_agent = Agent(
    role="Marketing and Communications Agent",
    goal="Effectively market the event and "
         "communicate with participants",
    tools=[search_tool, scrape_tool],
    verbose=True,
    backstory=(
        "Creative and communicative, "
        "you craft compelling messages and "
        "engage with potential attendees "
        "to maximize event exposure and participation."
    ),
    llm=llm
)

## Creating Venue Pydantic Object

- Create a class `VenueDetails` using [Pydantic BaseModel](https://docs.pydantic.dev/latest/api/base_model/).
- Agents will populate this object with information about different venues by creating different instances of it.

In [8]:
from pydantic import BaseModel
# Define a Pydantic model for venue details 
# (demonstrating Output as Pydantic)
class VenueDetails(BaseModel):
    name: str
    address: str
    capacity: int
    booking_status: str

## Creating Tasks
- By using `output_json`, you can specify the structure of the output you want.
- By using `output_file`, you can get your output in a file.
- By setting `human_input=True`, the task will ask for human feedback (whether you like the results or not) before finalising it.

In [9]:
venue_task = Task(
    description="Find a venue in {event_city} "
                "that meets criteria for {event_topic}.",
    expected_output="All the details of a specifically chosen"
                    "venue you found to accommodate the event.",
    human_input=True,
    output_json=VenueDetails,
    output_file="venue_details.json",  
      # Outputs the venue details as a JSON file
    agent=venue_coordinator
)

- By setting `async_execution=True`, it means the task can run in parallel with the tasks which come after it.

In [10]:
logistics_task = Task(
    description="Coordinate catering and "
                 "equipment for an event "
                 "with {expected_participants} participants "
                 "on {tentative_date}.",
    expected_output="Confirmation of all logistics arrangements "
                    "including catering and equipment setup.",
    human_input=True,
    async_execution=True,
    agent=logistics_manager
)

In [11]:
marketing_task = Task(
    description="Promote the {event_topic} "
                "aiming to engage at least"
                "{expected_participants} potential attendees.",
    expected_output="Report on marketing activities "
                    "and attendee engagement formatted as markdown.",
    # async_execution=True,
    output_file="marketing_report.md",  # Outputs the report as a text file
    agent=marketing_communications_agent
)

## Creating the Crew

**Note**: Since you set `async_execution=True` for `logistics_task` and `marketing_task` tasks, now the order for them does not matter in the `tasks` list.

In [12]:
# Define the crew with agents and tasks
event_management_crew = Crew(
    agents=[venue_coordinator, 
            logistics_manager, 
            marketing_communications_agent],
    
    tasks=[venue_task, 
           logistics_task, 
           marketing_task],
    
    verbose=True
)

## Running the Crew

- Set the inputs for the execution of the crew.

In [15]:
event_details = {
    'event_topic': "Tech Innovation Conference",
    'event_description': "A gathering of tech innovators "
                         "and industry leaders "
                         "to explore future technologies.",
    'event_city': "San Francisco",
    'tentative_date': "2025-09-15",
    'expected_participants': 500,
    'budget': 20000,
    'venue_type': "Conference Hall"
}

**Note 1**: LLMs can provide different outputs for they same input, so what you get might be different than what you see in the video.

**Note 2**: 
- Since you set `human_input=True` for some tasks, the execution will ask for your input before it finishes running.
- When it asks for feedback, use your mouse pointer to first click in the text box before typing anything.

In [16]:
result = event_management_crew.kickoff(inputs=event_details)

╭──────────────────────────────────────────── Crew Execution Started ─────────────────────────────────────────────╮
│                                                                                                                 │
│  Crew Execution Started                                                                                         │
│  Name: crew                                                                                                     │
│  ID: 19babf73-733c-4183-b324-b9dbbae0f9eb                                                                       │
│  Tool Args:                                                                                                     │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭─────────────────────────────────────────────── 🤖 Agent Started ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Agent: Venue Coordinator                                                                                       │
│                                                                                                                 │
│  Task: Find a venue in San Francisco that meets criteria for Tech Innovation Conference.                        │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭──────────────────────────────────────────── 🔧 Agent Tool Execution ────────────────────────────────────────────╮
│                                                                                                                 │
│  Agent: Venue Coordinator                                                                                       │
│                                                                                                                 │
│  Thought: Thought: To find a suitable venue in San Francisco for the Tech Innovation Conference, I need to      │
│  search for venues that meet the specified criteria, including location, capacity, and availability. The first  │
│  step is to search the internet for potential venues.                                                           │
│                                                                                                                 │
│  Using Tool: Search the internet with Serper                                                                    │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭────────────────────────────────────────────────── Tool Input ───────────────────────────────────────────────────╮
│                                                                                                                 │
│  "{\"search_query\": \"venues in San Francisco for tech conferences\"}"                                         │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭────────────────────────────────────────────────── Tool Output ──────────────────────────────────────────────────╮
│                                                                                                                 │
│  {'searchParameters': {'q': 'venues in San Francisco for tech conferences', 'type': 'search', 'num': 10,        │
│  'engine': 'google'}, 'organic': [{'title': 'Top 12 tech conferences in San Francisco for 2024 - TravelPerk',   │
│  'link': 'https://www.travelperk.com/blog/top-tech-conferences-in-san-francisco/', 'snippet': 'Top 12 tech      │
│  conferences in San Francisco for 2024 ; Developer Week. 21st - 23rd February 2024. Oakland Convention Center   │
│  ; Women Impact Tech. July ...', 'position': 1}, {'title': 'The Best 12 Tech Conferences to Attend in San       │
│  Francisco 2025', 'link': 'https://vanguard-x.com/tech-events-2025/tech-conferences-san-francisco-2025/',       │
│  'snippet': 'The AI Conference: Minds Meet in Mission Rock. Dates: September 17–18, 2025. Venue: Mission Rock,  │
│  San Francisco. Hosted by AI veterans ...', 'position': 2}, {'title': 'San Francisco - IAB Tech Lab', 'link':   │
│  'https://iabtechlab.com/venue/san-francisco-2/', 'snippet': 'Established in 2014, the IAB Technology           │
│  Laboratory (Tech Lab) is a non-profit consortium that engages a member community globally to develop           │
│  foundational ...', 'position': 3}, {'title': 'Venue PV CellTech USA - PV Tech Conferences', 'link':            │
│  'https://www.pvtechconferences.com/pv-celltech-usa/venue/', 'snippet': 'Enjoy our stunning waterfront venue    │
│  and easy access to San Francisco International Airport. Settle in to newly transformed accommodations.',       │
│  'position': 4}, {'title': 'Top Conference & Summit Venues in San Francisco, CA - PartySlate', 'link':          │
│  'https://www.partyslate.com/find-venues/conference-summit-venues/near/san-francisco-ca-usa', 'snippet': 'The   │
│  Melody of San Francisco. 906 Broadway, San Francisco, CA 94133, USA. Spaces for 120-350 Guests ;               │
│  InterContinental San Francisco. 888 Howard St, San Francisco ...', 'position': 5}, {'title': 'Best event       │
│  venues for a tech meetup : r/bayarea - Reddit', 'link':                                                        │
│  'https://www.reddit.com/r/bayarea/comments/1dt8ok5/best_event_venues_for_a_tech_meetup/', 'snippet': 'Hey      │
│  all, looking for suggestions including breweries/c...                                                          │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

...

╭────────────────────────────────────────────────── Tool Input ───────────────────────────────────────────────────╮
│                                                                                                                 │
│  "{\"website_url\": \"https://elev8.la/blog/how-to-create-a-conference-marketing-plan\"}"                       │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭────────────────────────────────────────────────── Tool Output ──────────────────────────────────────────────────╮
│                                                                                                                 │
│                                                                                                                 │
│  How to Create an Effective Conference Marketing Plan | Expert Guide                                            │
│  0                                                                                                              │
│  Skip to Content                                                                                                │
│  Home                                                                                                           │
│  Locations                                                                                                      │
│  Los Angeles                                                                                                    │
│  New York City                                                                                                  │
│  Atlanta                                                                                                        │
│  Houston                                                                                                        │
│  Washington, DC                                                                                                 │
│  Las Vegas                                                                                                      │
│  San Francisco                                                                                                  │
│  Miami                                                                                                          │
│  Chicago                                                                                                        │
│  Orange County                                                                                                  │
│  Orlando                                                                                                        │
│  New Orleans                                                                                                    │
│  San Diego                                                                                                      │
│  Austin                                                                                                         │
│  Dallas                                                                                                         │
│  San Jose                                                                                                       │
│  Boston                                                                                                         │
│  All Locations                                                                                                  │
│  Careers                                                                                                        │
│  Inquire                                                                                                        │
│  Open Menu                                                                                                      │
│  Close Menu                                                                                                     │
│  Home                                                                                                           │
│  Locations                                                                                                      │
│  Los Angeles                                                                                                    │
│  New York City                                                                                                  │
│  Atlanta                                              

╭───────────────────────────────────────────── ✅ Agent Final Answer ─────────────────────────────────────────────╮
│                                                                                                                 │
│  Agent: Marketing and Communications Agent                                                                      │
│                                                                                                                 │
│  Final Answer:                                                                                                  │
│  The text you've provided appears to be a comprehensive guide on how to plan and execute successful events,     │
│  specifically focusing on the staffing aspect. It highlights the importance of choosing the right event staff   │
│  for various roles such as bartenders, catering staff, brand ambassadors, check-in staff, production            │
│  assistants, and more across different cities like Los Angeles, New York City, Atlanta, Houston, Washington     │
│  D.C., Chicago, San Francisco, Miami, Las Vegas, and Orange County.                                             │
│                                                                                                                 │
│  To distill this information into actionable advice for planning a successful event:                            │
│                                                                                                                 │
│  1. **Define Event Objectives**: Clearly outline what you want to achieve with your event. This will help in    │
│  determining the type of staff you need.                                                                        │
│                                                                                                                 │
│  2. **Identify Staffing Needs**: Based on your objectives, decide which roles are essential (e.g., bartenders,  │
│  servers, greeters, brand ambassadors).                                                                         │
│                                                                                                                 │
│  3. **Choose Experienced Staff**: Opt for staff who have experience in events similar to yours. This ensures    │
│  they understand the dynamics and can handle unexpected situations.                                             │
│                                                                                                                 │
│  4. **Training is Key**: Even with experienced staff, providing some level of training or briefing on your      │
│  event's specifics can enhance performance and consistency.                                                     │
│                                                                                                                 │
│  5. **Consider Local Knowledge**: For events in different cities, consider hiring local staff who are familiar  │
│  with the area, laws, and cultural nuances.                                                                     │
│                                                                                                                 │
│  6. **Communicate Effectively**: Ensure that all staff members understand their roles, responsibilities, and    │
│  how they contribute to the overall success of the event.                                                       │
│                                                                                                                 │
│  7. **Logistics and Scheduling**: Plan meticulously, including scheduling, uniforms, equipment needs, and       │
│  contingency plans for last-minute changes or issues.                                                           │
│                                                                                                                 │
│  8. **Feedback Mechanism**: Have a system in place for 

╭──────────────────────────────────────────────── Task Completion ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Task Completed                                                                                                 │
│  Name: 6079153a-9c20-4e6a-ac74-654277e27e4f                                                                     │
│  Agent: Marketing and Communications Agent                                                                      │
│  Tool Args:                                                                                                     │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭──────────────────────────────────────────────── Crew Completion ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Crew Execution Completed                                                                                       │
│  Name: crew                                                                                                     │
│  ID: 19babf73-733c-4183-b324-b9dbbae0f9eb                                                                       │
│  Tool Args:                                                                                                     │
│  Final Output: The text you've provided appears to be a comprehensive guide on how to plan and execute          │
│  successful events, specifically focusing on the staffing aspect. It highlights the importance of choosing the  │
│  right event staff for various roles such as bartenders, catering staff, brand ambassadors, check-in staff,     │
│  production assistants, and more across different cities like Los Angeles, New York City, Atlanta, Houston,     │
│  Washington D.C., Chicago, San Francisco, Miami, Las Vegas, and Orange County.                                  │
│                                                                                                                 │
│  To distill this information into actionable advice for planning a successful event:                            │
│                                                                                                                 │
│  1. **Define Event Objectives**: Clearly outline what you want to achieve with your event. This will help in    │
│  determining the type of staff you need.                                                                        │
│                                                                                                                 │
│  2. **Identify Staffing Needs**: Based on your objectives, decide which roles are essential (e.g., bartenders,  │
│  servers, greeters, brand ambassadors).                                                                         │
│                                                                                                                 │
│  3. **Choose Experienced Staff**: Opt for staff who have experience in events similar to yours. This ensures    │
│  they understand the dynamics and can handle unexpected situations.                                             │
│                                                                                                                 │
│  4. **Training is Key**: Even with experienced staff, providing some level of training or briefing on your      │
│  event's specifics can enhance performance and consistency.                                                     │
│                                                                                                                 │
│  5. **Consider Local Knowledge**: For events in different cities, consider hiring local staff who are familiar  │
│  with the area, laws, and cultural nuances.                                                                     │
│                                                                                                                 │
│  6. **Communicate Effectively**: Ensure that all staff members understand their roles, responsibilities, and    │
│  how they contribute to the overall success of the event.                                                       │
│                                                                                                                 │
│  7. **Logistics and Scheduling**: Plan meticulously, including scheduling, uniforms, equipment needs, and       │
│  contingency plans for last-minute changes or issues.                                                           │
│                                                       

- Display the generated `venue_details.json` file.

In [17]:
import json
from pprint import pprint

with open('venue_details.json') as f:
   data = json.load(f)

pprint(data)

{'address': '123 Main St, San Francisco, CA',
 'booking_status': 'available',
 'capacity': 70000,
 'name': 'Conference Center'}


- Display the generated `marketing_report.md` file.

**Note**: After `kickoff` execution has successfully ran, wait an extra 45 seconds for the `marketing_report.md` file to be generated. If you try to run the code below before the file has been generated, your output would look like:

```
marketing_report.md
```

If you see this output, wait some more and than try again.

In [18]:
from IPython.display import Markdown
Markdown("marketing_report.md")

The text you've provided appears to be a comprehensive guide on how to plan and execute successful events, specifically focusing on the staffing aspect. It highlights the importance of choosing the right event staff for various roles such as bartenders, catering staff, brand ambassadors, check-in staff, production assistants, and more across different cities like Los Angeles, New York City, Atlanta, Houston, Washington D.C., Chicago, San Francisco, Miami, Las Vegas, and Orange County.

To distill this information into actionable advice for planning a successful event:

1. **Define Event Objectives**: Clearly outline what you want to achieve with your event. This will help in determining the type of staff you need.

2. **Identify Staffing Needs**: Based on your objectives, decide which roles are essential (e.g., bartenders, servers, greeters, brand ambassadors).

3. **Choose Experienced Staff**: Opt for staff who have experience in events similar to yours. This ensures they understand the dynamics and can handle unexpected situations.

4. **Training is Key**: Even with experienced staff, providing some level of training or briefing on your event's specifics can enhance performance and consistency.

5. **Consider Local Knowledge**: For events in different cities, consider hiring local staff who are familiar with the area, laws, and cultural nuances.

6. **Communicate Effectively**: Ensure that all staff members understand their roles, responsibilities, and how they contribute to the overall success of the event.

7. **Logistics and Scheduling**: Plan meticulously, including scheduling, uniforms, equipment needs, and contingency plans for last-minute changes or issues.

8. **Feedback Mechanism**: Have a system in place for attendees and staff to provide feedback during or after the event, which can be invaluable for future improvements.

9. **Legal and Ethical Considerations**: Ensure that all hiring practices comply with local labor laws and ethical standards.

10. **Review and Improve**: After each event, conduct a thorough review of what worked well and what didn’t, using this information to improve future events.

By following these steps and considering the specific needs of your event, you can ensure that your staffing choices contribute positively to the overall success of your event.